In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [4]:
df = pd.read_csv("iowa-sample.csv")
df.head()
# print(df["Item Description"].nunique())
# print(df["Item Number"].nunique())

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.50,0.13
1,S23102300041,12/20/2014,4346,Roy's Foodland,105 PEARL ST,SHELLSBURG,52332,POINT (-91.869285 42.094155),6,Benton,...,11774,Black Velvet,24,375,3.07,4.61,6,27.66,2.25,0.59
2,S14410500035,09/09/2013,3628,Wal-Mart 1528 / Cedar Rapids,2645 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.680734 42.034748),57,Linn,...,43410,Captain Morgan Parrot Bay Coconut,12,750,7.49,11.23,12,134.76,9.00,2.38
3,S09427600021,12/12/2012,4708,No Frills Supermarkets #803 / Glenwo,423 SHARP ST,GLENWOOD,51534,POINT (-95.742987 41.04635),65,Mills,...,24458,Kessler Blend Whiskey,6,1750,11.01,16.52,30,495.60,52.50,13.87
4,S28446900114,10/14/2015,2594,Hy-Vee Food Store / Sioux City,4500 SERGEANT ROAD,SIOUX CITY,51106,POINT (-96.346969 42.447396),97,Woodbury,...,10550,Black Velvet Toasted Caramel,12,750,6.75,10.13,12,121.56,9.00,2.38


In [ ]:
# Initial Feature Extration and Type Casting
df = pd.read_csv("iowa-sample.csv")

# DROP FEATURES THAT REPLICATE DATA
cols_to_drop = ["Invoice/Item Number", "Store Number", "Address", 
                "County Number", "Category", "Vendor Number", 
                "Volume Sold (Liters)", "Bottle Volume (ml)"]
df.drop(cols_to_drop, axis=1, inplace=True)


# TYPE CASTING
categorical_cols = ["Store Name", "City", "Zip Code", "Store Location",
                   "County", "Category Name", "Vendor Name"]
for col in categorical_cols:
    df[col] = df[col].astype('category')

# TIME MUNGING
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.insert(1, 'Year', df['Date'].dt.year)

df.head()
df.dtypes

In [ ]:
# Standardizing the Dataframe

def standardize(df): 
    #standardize only quantitative variables
    df_st = ((df.select_dtypes(float) - df.select_dtypes(float).mean()) / df.select_dtypes(float).std()) 

    #join the standardized quantites back with original df 
    df_st = df.select_dtypes(exclude=float).join(df_st)
    return df_st 

In [ ]:
class LR:
    
    def fit(self, X_train, y_train):
        # create vector of ones...
        ones = np.ones(shape=len(X_train))[..., None]
        #...and add to feature matrix
        X = np.concatenate((ones, X_train), 1)
        #calculate coefficients using closed-form solution
        self.coeffs = inv(X.transpose().dot(X)).dot(X.transpose()).dot(y_train)
        
    def predict(self, X_test):
        ones = np.ones(shape=len(X_test))[..., None]
        X_test = np.concatenate((ones, X_test), 1)
        y_hat = X_test.dot(self.coeffs)
        return y_hat


In [ ]:
from sklearn.linear_model import LinearRegression

data_dir = "https://dlsun.github.io/pods/data/"
bordeaux_df = pd.read_csv(data_dir + "bordeaux.csv",
                          index_col="year")

bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_test = bordeaux_df.loc[1980:].copy()

X_train = bordeaux_train[["age"]]
X_test = bordeaux_test[["age"]]
y_train = bordeaux_train["price"]

model = LinearRegression()
model.fit(X=X_train, y=y_train)
model.predict(X=X_test)

In [ ]:
model = LR()
model.fit(X_train, y_train)
model.predict(X_test)

In [ ]:
# EXTERNAL DATASET = IOWA POPULATION PER (COUNTY, YEAR)
df2 = pd.read_excel("iowa_county.xlsx")

# DATA CLEANING
df2 = df2.iloc[4:103]
df2.drop(["Unnamed: 1", "Unnamed: 2"], axis=1, inplace=True)
df2.rename(columns=
{"table with row headers in column A " +
 "and column headers in rows 3 through " +
 "4 (leading dots indicate sub-parts)": "County",
                    "Unnamed: 3": "2010", 
                    "Unnamed: 4": "2011", 
                    "Unnamed: 5": "2012", 
                    "Unnamed: 6": "2013", 
                    "Unnamed: 7": "2014",
                    "Unnamed: 8": "2015",
                    "Unnamed: 9": "2016",
                    "Unnamed: 10": "2017",
                    "Unnamed: 11": "2018",
                    "Unnamed: 12": "2019"}, inplace=True)
df2.reset_index(drop=True, inplace=True)
     
df2["County"] = df2["County"].str.replace(".", "")
df2["County"] = df2["County"].str.replace(" County, Iowa", "")

# CONVERT DF TO STANDARD FORM
df2 = df2.melt(var_name="Year", value_name="Population", id_vars=['County'])

# TYPE CASTING
df2["County"] = df2["County"].astype('category')
df2["Year"] = df2["Year"].astype('int')
df2["Population"] = df2["Population"].astype('int')
df2.head()

In [ ]:
# COMBINE IOWA LIQUOR WITH IOWA COUNTY TO GET POPULATION VALUES
df3 = pd.merge(df, df2, on=["County", "Year"])

# PER CAPITA NORMALIZATION
df3["Bottles Sold Per Capita"] = df3["Bottles Sold"]/df3["Population"]
df3["Alcohol Expense Per Capita"] = df3["Sale (Dollars)"]/df3["Population"]
df3["Volume Sold (Gallons) Per Capita"] = df3["Volume Sold (Gallons)"]/df3["Population"]

cols_to_drop = ["Bottles Sold", "Sale (Dollars)", "Volume Sold (Gallons)"]
df3.drop(cols_to_drop, axis=1, inplace=True)

df3.head()

In [ ]:
# EXTERNAL DATASET = IOWA INCOME BY COUNTY
df_income = pd.read_csv("income.csv")
#remove "IA" from County Name
mylist = []
for n in df_income["Name"]:
    name = n.split(",")
    name[0] = name[0].strip("")
    mylist.append(name[0])
    
df_income['County'] = mylist
# creating dataset with just county and avg income
income = df_income.groupby(('County'), as_index=False).agg({"Value": "mean"})
11:20
# COMBINE IOWA LIQUOR WITH IOWA INCOME TO GET POPULATION VALUES
df4 = pd.merge(df3, income, on='County')

In [ ]:
def k_fold(k, df):
    n = len(df)
    cut = int(n/k)
    folds = []
    start = 0
    end = cut
    for i in range(0, k):
        fold = df[start: end]
        folds.append(fold)
        start += int(n/k)
        end += int(n/k)
    return folds

def mse(actual, predicted):
    return -(((actual - predicted)**2).mean())

def cv(folds, response):
    test_errors = []
    for i in range(0, len(folds)):
        X_train = pd.DataFrame()
        Y_train = pd.Series(dtype=float)
        for j in range(0, len(folds)):
            if i == j:
                X_test = folds[j].drop([response], axis=1)
                Y_test = folds[j][response]
            if i != j:
                X_train = X_train.append(folds[j].drop([response], axis=1))
                Y_train = Y_train.append(folds[j][response])
    
        model = LR()
        model.fit(X_train, Y_train.transpose())
        Y_pred = pd.DataFrame()
        Y_pred = model.predict(X_test)
        error = mse(Y_test, Y_pred) 
        test_errors.append(error)
        rmse = np.sqrt(-sum(test_errors)/len(folds))
    return test_errors, rmse

In [ ]:
# SKLEARN CROSS VALIDATION
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

data_dir = "https://dlsun.github.io/pods/data/"
bordeaux_df = pd.read_csv(data_dir + "bordeaux.csv",
                          index_col="year")

bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_train["log(price)"] = np.log(bordeaux_train["price"])
bordeaux_train.drop(["price"], axis=1, inplace=True)

model = LinearRegression()
scores = cross_val_score(model, 
                         X=bordeaux_train[["win", "summer"]],
                         y=bordeaux_train["log(price)"],
                         scoring="neg_mean_squared_error",
                         cv=2)

print(scores)
print(np.sqrt(-scores).mean())

In [ ]:
# OUR CROSS VALIDATION 
bordeaux_train2 = bordeaux_train[["win", "summer", "log(price)"]]
folds = k_fold(2, bordeaux_train2)
errs, rmse = cv(folds, "log(price)")
print(errs)
print(rmse)
